<a href="https://colab.research.google.com/github/camille2019/cs6474_twitter_data_collection/blob/main/Twitter_Data_CS6474.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
!pip install twython


In [ ]:
credentials = {'api_key': 'ZusCICySM7BJsuqtHbUA6nj4A',
               'api_secret': 'RpcVyBtmLqLAAtCZIz0ObiW8PnTlaKZWYbo46CAlE2FA3EavR0',
               'access_token': '1291837066834145280-ebN9yj5u9MMNgIMUM34vGFGTR84pUY',
               'access_secret': 'wzmPJI1LYLD3sP19FfeCul0OUG4L0kOOaDEYaiekdXJ1K',
               'bearer_token': 'AAAAAAAAAAAAAAAAAAAAAJB8MgEAAAAAJhy8VF7YVX5I6FJujLzjDE7Y%2FVg%3DMvOkxbVIuyq2yVqLqmcbyp61XxCVTrhsZPez0dzsIxQ0SdijOb'}

with open('credentials.json', 'w') as f:
  json.dump(credentials, f, ensure_ascii=False)

In [ ]:
with open("credentials.json", "r") as file:
  creds = json.load(file)

In [ ]:
from twython import Twython


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
python_tweet = Twython(creds['access_token'], creds['access_secret'])

In [ ]:
def build_query(keyword, start_date):
  query = {'query': keyword,'start_time' : start_date}
  return query

In [ ]:
!pip install searchtweets

In [ ]:
from searchtweets import load_credentials
from searchtweets import gen_rule_payload
from searchtweets import ResultStream
import datetime


In [ ]:
def create_word_df(keyword, start_date, search_args):
  rule = gen_rule_payload(keyword, from_date=start_date, to_date=start_date)

  rs = ResultStream(rule_payload=rule,
                    max_results=500,
                    **search_args)
  tweets = rs.stream()
  tweet_text = []
  tweet_date = []
  tweet_id = []
  print(tweets)
  list_tweets = list(tweets)
  for tweet in list_tweets:
    tweet_text.append(tweet['text'])
    tweet_date.append(tweet['created_at'])
    tweet_id.append(tweet['id'])
  df = pd.DataFrame({'id':tweet_id,'tweet':tweet_text, 'date':tweet_date })
  return df


In [ ]:
#query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}
import requests
import os
import json

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


In [ ]:
import pandas as pd
import time

def sample_all_words( characters, character2date,keyword_sets, creds):
  # python_tweets = Twython(creds['api_key'], creds['api_secret'])
  search_url = "https://api.twitter.com/2/tweets/search/all"
  bearer_token = creds['bearer_token']
  df = pd.DataFrame()
  for char in characters:
    for word_set in keyword_sets:

      final_query = "(" + char + " " + word_set +")"
     # start, end = generate_random_date(start_date, end_date, s)
      announce_date = character_to_date[char]

      if "," in announce_date:
        announce_date_datetime = datetime.datetime.strptime(announce_date, '%B %d, %Y')
      else:
        announce_date_datetime = datetime.datetime.strptime(announce_date, '%B %Y')

      # TODO: adjust the timedelta frame for character where we don't have a specific start date
      end = announce_date_datetime + datetime.timedelta(hours=24)

      announce_date_datetime = announce_date_datetime.astimezone().isoformat()
      end = end.astimezone().isoformat()
  

  #first query is time movie/character was annouced
      query_params = {"query":final_query,'max_results': 150, 'start_time': announce_date_datetime, 'end_time':end, 'tweet.fields':['author_id,created_at,lang'] }#, 'lang':'en'}
      headers = create_headers(bearer_token)
      time.sleep(3)


      json_response = connect_to_endpoint(search_url, headers, query_params)
      print(json_response)

      if 'data' not in json_response:
        print('no response for' + " " + final_query)
        pass
      else:
        data = json_response['data']
        dict_ = {'tweet_id': [],'author_id': [], 'date': [], 'text': [], 'lang' :[], 'name_searched':[] }

        # # Search tweets
        # dict_ = {'user': [], 'date': [], 'text': [], 'tweet_id': []}
        for status in data:
            dict_['tweet_id'].append(status['id'])
            dict_['author_id'].append(status['author_id'])
            dict_['date'].append(status['created_at'])
            dict_['text'].append(status['text'])
            dict_['lang'].append(status['lang'])
            dict_['name_searched'].append(char)
        #print("dataframe so far")
        #print(df)
        df = df.append(pd.DataFrame(dict_))

#second query most recent tweets
      query_params = {"query":final_query,'max_results': 150, 'start_time': announce_date_datetime, 'end_time':end, 'tweet.fields':['author_id,created_at,lang']} #, 'lang':'en'}
      headers = create_headers(bearer_token)
      time.sleep(2)
      json_response = connect_to_endpoint(search_url, headers, query_params)
     # print(json_response)
      if 'data' not in json_response:
      #  print(" no response")
        print('no response for' + " " + final_query)
        pass
      else:

        data = json_response['data']
        dict_ = {'tweet_id': [],'author_id': [], 'date': [], 'text': [], 'lang':[], 'name_searched':[]}
       # print(data)
        # # Search tweets
        # dict_ = {'user': [], 'date': [], 'text': [], 'tweet_id': []}
        for status in data:
            dict_['tweet_id'].append(status['id'])
            dict_['author_id'].append(status['author_id'])
            dict_['date'].append(status['created_at'])
            dict_['text'].append(status['text'])
            dict_['lang'].append(status['lang'])
            dict_['name_searched'].append(char)
        df = df.append(pd.DataFrame(dict_))
        #print('dataframe so far')
        #print(df)
      
     # df = df.append(create_df)
  df.reset_index()
  return df

In [ ]:
# Margot Robbie/Harley Quinn, Idris Elba, and Tom Holland/Spider Man are repeated with different dates
# Still not sure what Storm version we're using?
# Also not sure where the hashtags go
character_list = ['Finn', 'John Boyega', 'Kylo Ren', 'Adam Driver', 'Rey', 'Daisy Ridley', 
                  'Abbie Mills', ' Nicole Beharie', 'Ichabod Crane', 'Tom Mison',
                  'Iris West', 'Candice Patton', 'Caitlin Snow', 'Danielle Panabaker',
                  'Idris Elba', '007', 'James Bond', 'Daniel Craig', 'Pierce Brosnan',
                  'Lashana Lynch', 'Nomi', '007', 'James Bond', 'Daniel Craig', 'Pierce Brosnan',
                  # Not sure if Spiderman and Spider Man need to be separate queries
                  'MJ', 'Mary Jane', 'Zendaya', 'Spider Man', 'Peter Parker', 'Tom Holland',
                  'Ariel', 'Little Mermaid', 'Halle Bailey', 'Angelina Jolie', 'Maleficent', 'Emma Watson', 'Cate Clanchett', 'Cinderella',
                  # Not sure if the "L." and "L" need to be separate queries
                  'Samuel L Jackson', 'Nick Fury', 'Jeremy Renner', 'Hawkeye', 'Mark Rufallo', 'Bruce Banner',
                  'Jurnee Smollett', 'Black Canary', 'Margot Robbie', 'Harley Quinn', 'Mary Elizabeth Winstead', 'The Huntress',
                  'Hamilton', 'Lin Manuel Miranda', 'Les Mis',
                  'Will Smith', 'Deadshot', 'Jared Leto', 'Joker', 
                  # 'Margot Robbie', 'Harley Quinn',
                  # 'Idris Elba', 
                  'Heimdall', 'Natlie Portman', 'Jane Foster',
                  'Tessa Thompson', 'Valkyrie', 'Scarlett Johannson', 'Black Widow',
                  'Angel Colby', 'Guinevere', 'Arthur', 'Bradley James',
                  'Zazie Beetz', 'Domino', 'Negasonic Teenage Warhead', 'Brianna Hildebrand',
                  'Donald Glover', 
                  # 'Spider Man', 'Tom Holland', 'Peter Parker', 
                  'Naomie Harris', 'Miss Moneypenny', 'Judi Dench', 'M', 'Vesper Lynd', 'Madeleine Swann', 'Eva Green', 'Léa Seydoux',
                  'Sam Wilson', ' Anthony Mackie', 'Bucky Barnes', 'Sebastian Stan',
                  'Anna Diop', 'Starfire', 'Teagan Croft', 'Raven',
                  'Pearl Mackie', 'Bill Potts', 'Jenna Coleman', 'Clara Oswald'
                  ]
character_to_date = {'Finn': 'April 29, 2014', 'John Boyega': 'April 29, 2014', 'Kylo Ren': 'April 29, 2014', 'Adam Driver': 'April 29, 2014', 'Rey': 'April 29, 2014', 'Daisy Ridley': 'April 29, 2014',
                     'Abbie Mills': 'September 16, 2013', ' Nicole Beharie': 'September 16, 2013', 'Ichabod Crane': 'September 16, 2013', 'Tom Mison': 'September 16, 2013',
                     'Iris West': 'February 4, 2014', 'Candice Patton': 'February 4, 2014', 'Caitlin Snow': 'February 4, 2014', 'Danielle Panabaker': 'February 4, 2014',
                     'Idris Elba': 'November 2008', '007': 'November 2008', 'James Bond': 'November 2008', 'Daniel Craig': 'November 2008', 'Pierce Brosnan': 'November 2008',
                     'Lashana Lynch': 'July 13, 2019', 'Nomi': 'July 13, 2019', '007': 'July 13, 2019', 'James Bond': 'July 13, 2019', 'Daniel Craig': 'July 13, 2019', 'Pierce Brosnan': 'July 13, 2019',
                     'MJ': 'August 2016', 'Mary Jane': 'August 2016', 'Zendaya': 'August 2016', 'Spider Man': 'August 2016', 'Peter Parker': 'August 2016', 'Tom Holland': 'August 2016',
                     'Ariel': 'July 2019', 'Little Mermaid': 'July 2019', 'Halle Bailey': 'July 2019', 'Angelina Jolie': 'July 2019', 'Maleficent': 'July 2019', 'Emma Watson': 'July 2019', 'Cate Clanchett': 'July 2019', 'Cinderella': 'July 2019',
                     'Samuel L Jackson': 'April 14, 2008', 'Nick Fury': 'April 14, 2008', 'Jeremy Renner': 'April 14, 2008', 'Hawkeye': 'April 14, 2008', 'Mark Rufallo': 'April 14, 2008', 'Bruce Banner': 'April 14, 2008',
                     'Jurnee Smollett': 'September 26, 2018', 'Black Canary': 'September 26, 2018', 'Margot Robbie': 'September 26, 2018', 'Harley Quinn': 'September 26, 2018', 'Mary Elizabeth Winstead': 'September 26, 2018', 'The Huntress': 'September 26, 2018',
                     'Hamilton': 'January 20, 2015', 'Lin Manuel Miranda': 'January 20, 2015', 'Les Mis': 'January 20, 2015',
                     'Will Smith': 'December 2014', 'Deadshot': 'December 2014', 'Jared Leto': 'December 2014', 'Joker': 'December 2014', 
                    #  'Margot Robbie': 'December 2014', 'Harley Quinn': 'December 2014',
                    #  'Idris Elba': 'November 2009', 
                     'Heimdall': 'November 2009', 'Natlie Portman': 'November 2009', 'Jane Foster': 'November 2009',
                     'Tessa Thompson': 'April 2016', 'Valkyrie': 'April 2016', 'Scarlett Johannson': 'April 2016', 'Black Widow': 'April 2016',
                     'Angel Colby': 'September 20, 2008', 'Guinevere': 'September 20, 2008', 'Arthur': 'September 20, 2008', 'Bradley James': 'September 20, 2008',
                     'Zazie Beetz': 'March 2017', 'Domino': 'March 2017', 'Negasonic Teenage Warhead': 'March 2017', 'Brianna Hildebrand': 'March 2017',
                     'Donald Glover': 'June 2017', 
                    #  'Spider Man': 'June 2017', 'Tom Holland': 'June 2017', 'Peter Parker': 'June 2017', 
                     'Naomie Harris': 'November 9, 2012', 'Miss Moneypenny': 'November 9, 2012', 'Judi Dench': 'November 9, 2012', 'M': 'November 9, 2012', 'Vesper Lynd': 'November 9, 2012', 'Madeleine Swann': 'November 9, 2012', 'Eva Green': 'November 9, 2012', 'Léa Seydoux': 'November 9, 2012',
                     'Sam Wilson': 'March 13, 2014', ' Anthony Mackie': 'March 13, 2014', 'Bucky Barnes': 'March 13, 2014', 'Sebastian Stan': 'March 13, 2014',
                     'Anna Diop': 'July 2018', 'Starfire': 'July 2018', 'Teagan Croft': 'July 2018', 'Raven': 'July 2018',
                     'Pearl Mackie': 'April 23, 2016', 'Bill Potts': 'April 23, 2016', 'Jenna Coleman': 'April 23, 2016', 'Clara Oswald': 'April 23, 2016'
                     }
keyword_set = [ '(racebend OR black OR white OR color OR divers OR politic OR rac)', 
               "(annoy OR remove OR hate OR never OR not OR dislike OR disturb OR threat OR scrutiniz OR humiliat OR angry OR rage OR pander OR nostalg OR scare OR trauma OR ignor OR notmy)",
               "(monkey OR gorilla OR persecut OR discriminat OR prejudice OR obscer OR unwelcome OR no OR hassle OR torment OR irritat OR aggravat OR victim OR aggress OR crime OR troll OR dominat)",
               "(marvel OR dc OR star wars OR skyfall OR bond OR hamilton OR merlin OR disney princess OR Les Mis)"
               ]


In [ ]:
character_to_date_test = {'Finn': 'April 29, 2014', 'John Boyega': 'April 29, 2014'}
character_list_test = ['Finn', 'John Boyega']

In [ ]:
full_data = sample_all_words(character_list, character_to_date, keyword_set, creds)

200
{'data': [{'id': '461259797468573697', 'author_id': '2432293886', 'lang': 'en', 'created_at': '2014-04-29T21:44:40.000Z', 'text': 'Beyond the army blanket, the Finn waited at the white table.'}, {'id': '461252098953576448', 'author_id': '14482363', 'lang': 'en', 'created_at': '2014-04-29T21:14:04.000Z', 'text': '♫ Just Announced: Austin, TX - Jul 18 at Black Finn Ameripub http://t.co/tKh6yLyXqA'}, {'id': '461233685052653568', 'author_id': '127956375', 'lang': 'en', 'created_at': '2014-04-29T20:00:54.000Z', 'text': "Finn's Reversible Bandana With Plastic Bag Pocket For Your Dog  Hot Tropical Color http://t.co/b6RPhCJQck via @Goodsmiths #handmadebot #dog"}, {'id': '461185379211968513', 'author_id': '281363481', 'lang': 'ht', 'created_at': '2014-04-29T16:48:57.000Z', 'text': 'RT @KiwhyAyiti: Grimel la Gon figi pizza ak pepperoni ki si ke soley Finn Rabi trop creme ansyen black'}, {'id': '461182429701865473', 'author_id': '139879473', 'lang': 'ht', 'created_at': '2014-04-29T16:37:14.00

In [ ]:
full_data

,tweet_id,author_id,date,text,lang,name_searched
0,461259797468573697,2432293886,2014-04-29T21:44:40.000Z,"Beyond the army blanket, the Finn waited at th...",en,Finn
1,461252098953576448,14482363,2014-04-29T21:14:04.000Z,"♫ Just Announced: Austin, TX - Jul 18 at Black...",en,Finn
2,461233685052653568,127956375,2014-04-29T20:00:54.000Z,Finn's Reversible Bandana With Plastic Bag Poc...,en,Finn
3,461185379211968513,281363481,2014-04-29T16:48:57.000Z,RT @KiwhyAyiti: Grimel la Gon figi pizza ak pe...,ht,Finn
4,461182429701865473,139879473,2014-04-29T16:37:14.000Z,Grimel la Gon figi pizza ak pepperoni ki si ke...,ht,Finn
...,...,...,...,...,...,...
12,723921116704677889,300195350,2016-04-23T17:06:58.000Z,A menos que se llame Clara Oswald no quiero sa...,es,Clara Oswald
13,723894547613245441,2961749268,2016-04-23T15:21:24.000Z,@DWPageOFFICIAL No actually I rather like Jen...,en,Clara Oswald
14,723853753267396608,365078788,2016-04-23T12:39:17.000Z,já pararam pra pensar que o doctor tem uma nov...,pt,Clara Oswald
15,723839100873220096,571738298,2016-04-23T11:41:04.000Z,"#IThinkTheNewCompanionIs Clara Oswald.\nNo, wa...",ht,Clara Oswald


In [ ]:
no_dupes = full_data.drop_duplicates(subset=['tweet_id'])


In [ ]:
no_dupes

,tweet_id,author_id,date,text,lang,name_searched
0,461259797468573697,2432293886,2014-04-29T21:44:40.000Z,"Beyond the army blanket, the Finn waited at th...",en,Finn
1,461252098953576448,14482363,2014-04-29T21:14:04.000Z,"♫ Just Announced: Austin, TX - Jul 18 at Black...",en,Finn
2,461233685052653568,127956375,2014-04-29T20:00:54.000Z,Finn's Reversible Bandana With Plastic Bag Poc...,en,Finn
3,461185379211968513,281363481,2014-04-29T16:48:57.000Z,RT @KiwhyAyiti: Grimel la Gon figi pizza ak pe...,ht,Finn
4,461182429701865473,139879473,2014-04-29T16:37:14.000Z,Grimel la Gon figi pizza ak pepperoni ki si ke...,ht,Finn
...,...,...,...,...,...,...
11,723921141941850117,464288496,2016-04-23T17:07:04.000Z,"Forget the new companion, no one can beat rose...",en,Clara Oswald
12,723921116704677889,300195350,2016-04-23T17:06:58.000Z,A menos que se llame Clara Oswald no quiero sa...,es,Clara Oswald
14,723853753267396608,365078788,2016-04-23T12:39:17.000Z,já pararam pra pensar que o doctor tem uma nov...,pt,Clara Oswald
15,723839100873220096,571738298,2016-04-23T11:41:04.000Z,"#IThinkTheNewCompanionIs Clara Oswald.\nNo, wa...",ht,Clara Oswald


In [ ]:
english_only = no_dupes[no_dupes['lang']=='en']
english_only

,tweet_id,author_id,date,text,lang,name_searched
0,461259797468573697,2432293886,2014-04-29T21:44:40.000Z,"Beyond the army blanket, the Finn waited at th...",en,Finn
1,461252098953576448,14482363,2014-04-29T21:14:04.000Z,"♫ Just Announced: Austin, TX - Jul 18 at Black...",en,Finn
2,461233685052653568,127956375,2014-04-29T20:00:54.000Z,Finn's Reversible Bandana With Plastic Bag Poc...,en,Finn
5,461146329264046081,604765876,2014-04-29T14:13:47.000Z,"Finn 2, a long haired black cheese boy, and Br...",en,Finn
6,461141522730459138,12373992,2014-04-29T13:54:41.000Z,@Finn_Mackay @fusillisara being a woman may no...,en,Finn
...,...,...,...,...,...,...
10,723891204312698881,624705452,2016-04-23T15:08:07.000Z,"""Tell me you will never send me away ever agai...",en,Clara Oswald
11,723848430057975808,383578435,2016-04-23T12:18:08.000Z,Ill be asleep when they announce the new compa...,en,Clara Oswald
12,723688465565503488,1242317940,2016-04-23T01:42:30.000Z,Something along the why is my sit stand desk w...,en,Clara Oswald
2,723965636678967296,287677341,2016-04-23T20:03:53.000Z,no one will replace my true love clara oswald ...,en,Clara Oswald


In [ ]:
english_only.to_csv("character_tweets_english.csv")